In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
import numpy as np
import operator
from types import SimpleNamespace

cot = lambda x: 1 / np.tan(x)

def nu_arg(p):
    num = p.E_z * np.exp(1j * p.theta) - 1j * p.alpha * p.k_x
    denom = p.E_z**2 + p.alpha * p.k_x * (p.alpha * p.k_x - 2 * p.E_z * np.sin(p.theta))
    return num / np.sqrt(denom)

def omega(p, j, sign):
    op = operator.sub if sign < 0 else operator.add
    return 0.25 * op(gamma(p, j), delta(p, j))

def gamma(p, j):
    k = k_j(p, j)
    x = 1j * k * np.tan(p.W * k / 2)
    return (q(p) + x) / (q(p) - x)

def delta(p, j):
    k = k_j(p, j)
    x = 1j * k * cot(p.W * k / 2)
    return (q(p) - x) / (q(p) + x)

def beta(p, sign):
    w_1 = omega(p, 1, sign)
    w_2 = omega(p, 2, sign)
    nu = nu_arg(p)
    beta = np.array([
        [nu * (w_1 - w_2), (w_1 + w_2)],
        [-(w_1 + w_2), nu.conj() * (w_2 - w_1)]
    ])
    return beta

def S(p):
    p.E_z = p.mu_B * p.g * p.B_x
    r_ll = beta(p, +1)
    t_rl = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_lr = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_rr = np.exp(-2j * q(p) * p.W) * beta(p, +1)

    S = np.block([[r_ll, t_rl],
                  [t_lr, t_rr]])
    return S

def q(p):
    return np.sqrt((2 * p.m_eff / p.hbar**2) * p.mu_n - p.k_x**2)

def k_j(p, j):
    op = operator.sub if j == 1 else operator.add
    sqrt_term = np.sqrt(
        p.E_z**2
        - 2 * p.alpha * p.E_z * np.sin(p.theta) * p.k_x + p.alpha**2 * p.k_x**2)
    k_sq = (2 * p.m_eff / p.hbar**2) * op(p.mu_n, sqrt_term) - p.k_x**2
    return np.sqrt(k_sq)

k_F = lambda p: np.sqrt(p.mu_n * (2 * p.m_eff)) / p.hbar

In [ ]:
import scipy.constants
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (
        scipy.constants.eV * 1e-3))

params = SimpleNamespace(g=50, mu_n=10, alpha=20, B_x=1, theta=0, k_x=0.03, W=100, phi=np.pi, **constants)

S(params)

## Bas's implementation

In [ ]:
def energy_operator(params):
    import cmath
    if params.k_x == 0:
        smat_plus = smat_min = S(p=params)
    else:
        params.k_x *= -1
        smat_min = S(p=params)
        params.k_x *= -1
        smat_plus = S(p=params)

    phase = cmath.exp(1j * params.phi / 2)
    r = np.diag([phase, phase, phase.conjugate(), phase.conjugate()])
    smat_prod = smat_plus.T.conj() @ r @ smat_min.T @ r.conj()
    return 0.5 * np.eye(smat_prod.shape[0]) - 0.25 * (smat_prod + smat_prod.T.conj())

def energies_over_delta(params):
    """Same as energy_operator(), but returns the
    square-root of the eigenvalues"""
    operator = energy_operator(params)
    return np.sqrt(np.linalg.eigvalsh(operator))

In [ ]:
def plot(B_x, alpha, mu_n, theta, phi, params=params):
    setattr(params, 'B_x', B_x)
    setattr(params, 'alpha', alpha)
    setattr(params, 'mu_n', mu_n)
    setattr(params, 'theta', theta)
    setattr(params, 'phi', phi)
    
    k_max = 0.92*k_F(params)
    ks = np.linspace(-k_max, k_max, 150)
    Es = [energies_over_delta(params) for params.k_x in ks]
    return hv.Path((ks, Es))[:, 0:1.1]

hv.DynamicMap(plot, kdims=['B_x', 'alpha', 'mu_n', 'theta', 'phi']).redim.range(
    B_x=(0.01, 1), alpha=(1, 100), mu_n=(10, 50), theta=(0, np.pi), phi=(0, np.pi))

## Tom's implementation

In [ ]:
def energy_operator(params):
    import cmath
    if params.k_x == 0:
        smat_plus = smat_min = S(p=params)
    else:
        params.k_x *= -1
        smat_min = S(p=params)
        params.k_x *= -1
        smat_plus = S(p=params)

    phase = cmath.exp(1j * params.phi / 2)
    r = np.diag([phase, phase, phase.conjugate(), phase.conjugate()])
    smat_prod = r @ smat_min.conj() @ r.conj() @ smat_plus
    return smat_prod


def energies_over_delta(syst):
    """Same as energy_operator(), but returns the
    square-root of the eigenvalues"""
    from numpy.lib.scimath import sqrt
    operator = energy_operator(syst)
    es = -np.linalg.eigvals(operator)
    alpha = np.hstack((sqrt(es), -sqrt(es)))
    alpha = alpha[np.imag(alpha)<=0]

    return np.sort(np.real((1+alpha**2)/(2*alpha)))
ks = np.linspace(0.0, 0.06, 50)
Es = [energies_over_delta(params) for params.k_x in ks]
hv.Path((ks, Es))